In [1]:
# check the model of using differnet window size, compare their results on valid dataset
#    test dataset without any preprocessing, no scaling and no normalization
# decide which window size is the optimal

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from torch.autograd import Variable

import my_model
from utilities import MyTrainDataSet, MyTestDataSet, load_data_2, load_test_data, min_max_scaling, normalize_one, construct_train_valid_tensor, construct_test_tensor, show_statistic

Plot result

In [3]:
def getDistanceThetaPhi(xyz): # N x 3 -> N x 2, xyz_tensor must be two dimension
    size = xyz.shape[0]
    dtp = np.zeros([size, 3])
    for i in range(size):
        x = xyz[i, 0]
        y = xyz[i, 1]
        z = xyz[i, 2]
        # print(z)
        d = math.sqrt(x*x + y*y + z*z)
        theta = math.atan(math.sqrt(x*x + y*y)/z)/math.pi*180
        phi = math.atan(y/x)/math.pi*180
        
        if (d >= 2.0):
            d= 2.0;
        elif (d <= 1.0):
            d= 1.0;
        
        if (theta < 0):
            theta = 180 + theta;
        
        if (phi > 0):
            if (x > 0):
                phi = phi;
            else:
                phi = 180 + phi;
        else:
            if (x < 0):
                phi = 180 + phi;
            else:
                phi = 360 + phi;
        
        
        dtp[i, 0] = d
        dtp[i, 1] = theta
        dtp[i, 2] = phi
    
    return dtp   

In [4]:
input_valid_train = np.load("input_valid_tensor.npy")
gt_valid_label = np.load("gt_valid_label_tensor.npy")
pd_valid_label = np.load("pd_valid_label_tensor.npy")

# construct 2d theta phi valid tensor
gt_dthetaphi_valid_label = getDistanceThetaPhi(gt_valid_label)
pd_dthetaphi_valid_label = getDistanceThetaPhi(pd_valid_label)
gt_thetaphi_valid_label_tensor = torch.Tensor(gt_dthetaphi_valid_label[:,1:])
pd_thetaphi_valid_label_tensor = torch.Tensor(pd_dthetaphi_valid_label[:,1:])
print("gt_thetaphi_valid_label_tensor shape:", gt_thetaphi_valid_label_tensor.shape)
print("pd_thetaphi_valid_label_tensor shape:", pd_thetaphi_valid_label_tensor.shape)

gt_thetaphi_valid_label_tensor = torch.matmul(gt_thetaphi_valid_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))
pd_thetaphi_valid_label_tensor = torch.matmul(pd_thetaphi_valid_label_tensor, torch.Tensor([[1/180, 0],[0, 1/360]]))

gt_thetaphi_valid_label_tensor shape: torch.Size([397, 2])
pd_thetaphi_valid_label_tensor shape: torch.Size([397, 2])


In [5]:
class MDN(nn.Module):
    def __init__(self, n_input, n_hidden, n_gaussians):
        super(MDN, self).__init__()
        self.l_h = nn.Sequential(
            nn.Linear(n_input, n_hidden),
            nn.Tanh()
        )
        self.l_pi = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_theta = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_theta = nn.Linear(n_hidden, n_gaussians)
        
        self.l_mu_phi = nn.Linear(n_hidden, n_gaussians)
        self.l_sigma_phi = nn.Linear(n_hidden, n_gaussians)
        
        
        self.l_correlation_theta_phi = nn.Sequential(
            nn.Linear(n_hidden, n_gaussians),
            nn.Tanh()
        )
        
    def forward(self, x):
        h = self.l_h(x)
        # print("h", h.shape)
        # print("h[0]", h[0, :])
        
        pi = F.softmax(self.l_pi(h), -1)
        
        # print("pi", pi.shape)
        # print("pi[0]", pi[0, :])
        mu_theta = self.l_mu_theta(h)
        # print("mu_theta", pi.shape)
        mu_phi = self.l_mu_phi(h)
        # print("mu_phi", pi.shape)
        
        # use exp to ensure positive range
        sigma_theta = torch.exp(self.l_sigma_theta(h))
        # print("sigma_theta", sigma_theta.shape)
        sigma_phi = torch.exp(self.l_sigma_phi(h))
        # print("sigma_phi", sigma_phi.shape)

        # use tanh to ensoure range of (-1, 1)
        correlation_theta_phi = self.l_correlation_theta_phi(h)
        # print("correlation_y_z", pi.shape)
        # print("correlation_y_z[0]", correlation_y_z[0, :])
        
        return pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi


In [6]:
def mdn_loss_fn(y, pi, mu_theta, mu_phi, sigma_theta, sigma_phi, correlation_theta_phi):
    size = y.shape[0]
    n_gaussians = pi.shape[1]
    '''
    print("sample size: ", size)
    print("num of gaus: ", n_gaussians)
    print("mu_theta size: ", mu_theta.shape)
    print("mu_phi size: ", mu_phi.shape)
    print("sigma_theta size: ", sigma_theta.shape)
    print("sigma_phi size: ", sigma_phi.shape)
    print("correlation_theta_phi size: ", correlation_theta_phi.shape)
    '''
    # build mean matrix
    mean = torch.stack((mu_theta, mu_phi), dim=2)
    # build covariance matrix with standard bivariate normal distribution
    cov = torch.zeros([size, n_gaussians, 2, 2])
    cov[:, :, 0, 0] = sigma_theta**2
    cov[:, :, 1, 1] = sigma_phi**2
    cov[:, :, 1, 0] = correlation_theta_phi*sigma_theta*sigma_phi
    cov[:, :, 0, 1] = correlation_theta_phi*sigma_theta*sigma_phi

    
    '''
    print("sigma_theta**2 size: ", (sigma_theta**2).shape)
    print("sigma_phi**2 size: ", (sigma_phi**2).shape)
    print("correlation_theta_phi*sigma_theta*sigma_phi size: ", (correlation_theta_phi*sigma_theta*sigma_phi).shape)
    print("mean size:", mean.shape)
    print("cov size:", cov.shape)
    print("correlation_theta_phi:", correlation_theta_phi.min())
    '''
    # mean: N x n_gaussian x 2
    # cov: N x 5 x 2 x 2
    m = torch.distributions.multivariate_normal.MultivariateNormal(loc=mean, covariance_matrix=cov)
    # expand y to N x n_gaussian x 2
    y = y.unsqueeze(1).repeat(1, n_gaussians, 1)
    
    # print("y size", y.shape) # 14985 x 5 x 2
    
    # y: [x, y, z]; row: x, y, z; column: N, number of samples
    # y: N x 3
    # loss: N x 1
    # print(y_normal.is_cuda)
    likelihood = torch.exp(m.log_prob(y))
    
    # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    # print("pi shape: ", pi.shape) # 14985 x 5
    loss = torch.sum(likelihood * pi, dim=1) # 14985 X 1
    # print(likelihood.max())
    '''
    for i in range(loss.shape[0]):
        if loss[i] > 1.5:
            print("loss: ", loss[i])
            print("likelihood: ", likelihood[i])
            print("mu array  : ", pi[i])
            # print("likelihood shape: ", likelihood.shape) # 14985 x 5
    '''
    loss = -torch.log(loss)
    return torch.mean(loss)

In [53]:
test_3_loss = []
for i in range(2, 6):
    num_gaussians = i
    PATH = "model/with_early_stop_after_400_without_normalization/model_ng_" + str(i) + ".pt"

    load_model_mdn = MDN(2, n_hidden=20, n_gaussians=num_gaussians)
    load_model_mdn.load_state_dict(torch.load(PATH))
    load_model_mdn.eval()
    # mmodel = torch.load(PATH)
    
    # Calculating validation data loss
    pi_valid, mu_theta_valid, mu_phi_valid, sigma_theta_valid, sigma_phi_valid, correlation_theta_phi_valid = load_model_mdn(pd_thetaphi_valid_label_tensor)
    loss_valid = mdn_loss_fn(gt_thetaphi_valid_label_tensor,
                             pi_valid, mu_theta_valid,
                             mu_phi_valid,
                             sigma_theta_valid,
                             sigma_phi_valid,
                             correlation_theta_phi_valid)
    print(loss_valid.data.item())
    
    test_3_loss.append(loss_valid.data.item())

-4.260011196136475
-4.889506816864014
-4.778032302856445
-4.788808345794678


In [16]:
x = ['2', '3', '4', '5']

x_pos = [i for i, _ in enumerate(x)]

# plt.bar(x_pos, test_1_loss)
plt.bar(x_pos, test_3_loss, color=['tab:blue',
                                   'tab:blue',
                                   'tab:green',
                                   'tab:blue'], zorder = 3)

plt.grid(zorder=0)

plt.xlabel("Input Window Size", fontsize=12)
plt.ylabel("MSE Loss", fontsize=12)
plt.xticks(x_pos, x)


# plt.ylim([0.025,0.039])
# plt.show()

([<matplotlib.axis.XTick at 0x7f083f0710b8>,
 <a list of 4 Text xticklabel objects>)

In [10]:
test_3_loss

[-4.260011196136475,
 -4.889506816864014,
 -4.778032302856445,
 -4.788808345794678]

In [60]:
x = ['2', '3', '4', '5']

# x_pos = [i for i, _ in enumerate(x)]

plt.plot(x, test_3_loss, c='tab:blue')
plt.scatter(x[1], test_3_loss[1], marker='s', s=250, c='tab:green')
plt.grid(zorder=0)






plt.xlabel("Number of Bivariate Normal Distribution Used for Gaussian Mixture", fontsize=12)
plt.ylabel("Negative Log Likelihood", fontsize=12)
# plt.xticks(x_pos, x)

Text(31.097222222222214, 0.5, 'Negative Log Likelihood')